# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"
import cupy as cp

In [4]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [5]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [8]:
FONTSIZE = 20

In [9]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

# Functions

In [10]:
def generate_pairs(lst):
    """
    Generates all unique pairs from a list.

    Parameters:
    - lst (list): The list to generate pairs from.

    Returns:
    - list: A list of tuples, each containing a unique pair from the input list.
    """
    n = len(lst)
    return [(lst[i], lst[j]) for i in range(n) for j in range(i+1, n)]

In [11]:
def update_array_by_mask(array, mask, value=np.nan):
    """
    Update elements of an array based on a mask and replace them with a specified value.

    Parameters:
    - array (np.array): The input numpy array whose values are to be updated.
    - mask (np.array): A boolean array with the same shape as `array`. Elements of `array` corresponding to True in the mask are replaced.
    - value (scalar, optional): The value to assign to elements of `array` where `mask` is True. Defaults to np.nan.

    Returns:
    - np.array: A copy of the input array with updated values where the mask is True.

    Example:
    >>> array = np.array([1, 2, 3, 4])
    >>> mask = np.array([False, True, False, True])
    >>> update_array_by_mask(array, mask, value=0)
    array([1, 0, 3, 0])
    """
    result = array.copy()
    result[mask] = value
    return result

In [12]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        # linear interpolation of NaNs
        nans, x= nan_helper(y)
        y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [13]:
def interpolate_signal(signal):
    """
    Interpolates missing values (NaNs) in a given signal array using linear interpolation.

    The function finds NaN elements in the signal, computes linear interpolation based on
    non-NaN values, and fills the NaNs with these interpolated values.

    Parameters:
        signal (numpy.ndarray): The input signal array containing NaN values that need to be interpolated.

    Returns:
        numpy.ndarray: The signal array with NaN values interpolated.
    """
    if signal is None or not isinstance(signal, np.ndarray):
        raise ValueError("Input must be a numpy ndarray.")
    
    result = signal.copy()
    nans, x = nan_helper(result)
    result[nans] = np.interp(x(nans), x(~nans), result[~nans])
    return result

In [14]:
def recursive_defaultdict():
    return defaultdict(recursive_defaultdict)

## Inputs & Data

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [16]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5
RESAMPLE_RATE=1000

In [17]:
zscore_threshold = 4
VOLTAGE_SCALING_VALUE = 0.195

In [18]:
BAND_TO_FREQ = {"theta": (4,12), "gamma": (30,51)}

In [19]:
LFP_TRACES_DF = pd.read_pickle("./proc/rce_pilot_2_01_lfp_traces_and_frames.pkl")

In [20]:
LFP_TRACES_DF.shape

(61, 23)

## Preprocessing

In [21]:
original_trace_columns = [col for col in LFP_TRACES_DF.columns if "trace" in col]

In [22]:
original_trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [23]:
for col in original_trace_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float32) * VOLTAGE_SCALING_VALUE)

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [24]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[95.354996, 82.09499, 97.5, 132.405, 123.825, ...","[46.019997, 49.335, 75.27, 97.89, 77.61, 40.55...","[61.425, 66.104996, 81.899994, 90.479996, 71.5...","[54.6, 54.405, 73.32, 86.189995, 59.085, 19.89...","[55.574997, 79.365, 128.11499, 170.43, 189.344..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[95.354996, 82.09499, 97.5, 132.405, 123.825, ...","[46.019997, 49.335, 75.27, 97.89, 77.61, 40.55...","[61.425, 66.104996, 81.899994, 90.479996, 71.5...","[54.6, 54.405, 73.32, 86.189995, 59.085, 19.89...","[55.574997, 79.365, 128.11499, 170.43, 189.344..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[19.89, 29.445, 33.149998, 37.829998, 45.43499...","[29.445, 28.859999, 25.935, 23.205, 21.449999,...","[28.47, 25.349998, 22.035, 22.814999, 23.00999...","[68.64, 90.284996, 93.795, 71.564995, 90.09, 1...","[62.984997, 86.774994, 104.13, 86.96999, 75.65..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[19.89, 29.445, 33.149998, 37.829998, 45.43499...","[29.445, 28.859999, 25.935, 23.205, 21.449999,...","[28.47, 25.349998, 22.035, 22.814999, 23.00999...","[68.64, 90.284996, 93.795, 71.564995, 90.09, 1...","[62.984997, 86.774994, 104.13, 86.96999, 75.65..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 24

# Calculating modified zscore

`0.6745(xi – x̃) / MAD`

In [25]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_MAD".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: stats.median_abs_deviation(x))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [26]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_modified_zscore".format(brain_region)
    MAD_column = "{}_lfp_MAD".format(brain_region)

    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF.apply(lambda x: 0.6745 * (x[col] - np.median(x[col])) / x[MAD_column], axis=1)

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [27]:
LFP_TRACES_DF[updated_column]

0     [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
1     [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
2     [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
3     [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
4     [0.27719176, 0.40500796, 0.35572946, -0.004619...
                            ...                        
56    [0.22029124, 0.7880522, 1.144606, 0.71083665, ...
57    [0.20513144, 0.031291235, -0.13791321, -0.1900...
58    [0.20513144, 0.031291235, -0.13791321, -0.1900...
59    [-1.0707098, -1.1721206, -1.1367447, -0.910339...
60    [-1.0707098, -1.1721206, -1.1367447, -0.910339...
Name: vHPC_lfp_modified_zscore, Length: 61, dtype: object

## calculating root mean sequare

In [28]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: (x / np.sqrt(np.mean(x**2))).astype(np.float32))


mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [29]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC_lfp_modified_zscore,MD_lfp_modified_zscore,LH_lfp_modified_zscore,BLA_lfp_modified_zscore,vHPC_lfp_modified_zscore,mPFC_lfp_RMS,MD_lfp_RMS,LH_lfp_RMS,BLA_lfp_RMS,vHPC_lfp_RMS
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.89292616, 0.77410233, 0.9121477, 1.2249339,...","[0.47153494, 0.50638753, 0.77905774, 1.0168754...","[0.94851553, 1.0207834, 1.2646874, 1.3971784, ...","[0.66973317, 0.6673498, 0.8985389, 1.0558428, ...","[0.23408099, 0.3398509, 0.5565925, 0.7447243, ...","[0.19477782, 0.16769215, 0.19915934, 0.2704583...","[0.13429306, 0.14396672, 0.21964882, 0.2856573...","[0.22216085, 0.23908737, 0.29621443, 0.3272464...","[0.15934613, 0.15877703, 0.2139791, 0.25153923...","[0.09461661, 0.13511916, 0.21811618, 0.2901576..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.89292616, 0.77410233, 0.9121477, 1.2249339,...","[0.47153494, 0.50638753, 0.77905774, 1.0168754...","[0.94851553, 1.0207834, 1.2646874, 1.3971784, ...","[0.66973317, 0.6673498, 0.8985389, 1.0558428, ...","[0.23408099, 0.3398509, 0.5565925, 0.7447243, ...","[0.19477782, 0.16769215, 0.19915934, 0.2704583...","[0.13429306, 0.14396672, 0.21964882, 0.2856573...","[0.22216085, 0.23908737, 0.29621443, 0.3272464...","[0.15934613, 0.15877703, 0.2139791, 0.25153923...","[0.09461661, 0.13511916, 0.21811618, 0.2901576..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2918245, 0.42672452, 0.4790326, 0.5451061, ...","[0.8516992, 0.83455086, 0.7488094, 0.66878396,...","[0.84312505, 0.75009054, 0.6512414, 0.6745, 0....","[0.86968505, 1.1439322, 1.1884048, 0.9067454, ...","[0.93568933, 1.2858553, 1.5413042, 1.2887255, ...","[0.25848255, 0.38265556, 0.43080425, 0.4916236...","[0.6991664, 0.6852757, 0.6158221, 0.5509987, 0...","[0.6525567, 0.5810436, 0.50506103, 0.52293926,...","[0.83570427, 1.099236, 1.1419709, 0.8713166, 1...","[0.88643956, 1.2212558, 1.4655069, 1.2240001, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2918245, 0.42672452, 0.4790326, 0.5451061, ...","[0.8516992, 0.83455086, 0.7488094, 0.66878396,...","[0.84312505, 0.75009054, 0.6512414, 0.6745, 0....","[0.86968505, 1.1439322, 1.1884048, 0.9067454, ...","[0.93568933, 1.2858553, 1.5413042, 1.2887255, ...","[0.25848255, 0.38265556, 0.43080425, 0.4916236...","[0.6991664, 0.6852757, 0.6158221, 0.5509987, 0...","[0.6525567, 0.5810436, 0.50506103, 0.52293926,...","[0.83570427, 1.099236, 1.1419709, 0.8713166, 1...","[0.88643956, 1.2212558, 1.4655069, 1.2240001, ..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[1

## Filtering for zscore value

In [30]:
zscore_columns = [col for col in LFP_TRACES_DF.columns if "zscore" in col]

In [31]:
zscore_columns

['mPFC_lfp_modified_zscore',
 'MD_lfp_modified_zscore',
 'LH_lfp_modified_zscore',
 'BLA_lfp_modified_zscore',
 'vHPC_lfp_modified_zscore']

In [32]:
for col in zscore_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_mask".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: np.abs(x) >= zscore_threshold)

mPFC_lfp_modified_zscore
MD_lfp_modified_zscore
LH_lfp_modified_zscore
BLA_lfp_modified_zscore
vHPC_lfp_modified_zscore


In [33]:
LFP_TRACES_DF[updated_column].head()

0    [False, False, False, False, False, False, Fal...
1    [False, False, False, False, False, False, Fal...
2    [False, False, False, False, False, False, Fal...
3    [False, False, False, False, False, False, Fal...
4    [False, False, False, False, False, False, Fal...
Name: vHPC_lfp_mask, dtype: object

In [34]:
LFP_TRACES_DF[updated_column].iloc[0].shape

(3414668,)

In [35]:
sum(LFP_TRACES_DF[updated_column].iloc[0])

120878

- Filtering raw traces by zscore

In [36]:
LFP_TRACES_DF[col].head()

0    [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
1    [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
2    [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
3    [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
4    [0.27719176, 0.40500796, 0.35572946, -0.004619...
Name: vHPC_lfp_modified_zscore, dtype: object

In [37]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_trace_filtered".format(brain_region)    
    mask_column = "{}_lfp_mask".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF.apply(lambda x: update_array_by_mask(x[col], x[mask_column]), axis=1)

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [38]:
LFP_TRACES_DF[col].head()

0    [55.574997, 79.365, 128.11499, 170.43, 189.344...
1    [55.574997, 79.365, 128.11499, 170.43, 189.344...
2    [62.984997, 86.774994, 104.13, 86.96999, 75.65...
3    [62.984997, 86.774994, 104.13, 86.96999, 75.65...
4    [34.32, 50.504997, 44.265, -1.365, -31.784998,...
Name: vHPC_lfp_trace, dtype: object

In [39]:
sum(np.isnan(LFP_TRACES_DF[col].iloc[0]))

0

- Calculating RMS of filtered signal

In [40]:
filtered_trace_column = [col for col in LFP_TRACES_DF if "lfp_trace_filtered" in col]

In [41]:
for col in filtered_trace_column:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS_filtered".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: (x / np.sqrt(np.nanmean(x**2))).astype(np.float32))

mPFC_lfp_trace_filtered
MD_lfp_trace_filtered
LH_lfp_trace_filtered
BLA_lfp_trace_filtered
vHPC_lfp_trace_filtered


- Dropping unnecessary columns

In [42]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "zscore" in col or "MAD" in col], errors="ignore")

In [43]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "RMS" in col and not "filtered" in col], errors="ignore")


In [44]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "mask" in col], errors="ignore")


In [45]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "trace" in col], errors="ignore")


In [46]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_RMS_filtered', 'MD_lfp_RMS_filtered',
       'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered', 'vHPC_lfp_RMS_filtered'],
      dtype='object')

- Interpolating filtered RMS

In [47]:
filtered_RMS_column = [col for col in LFP_TRACES_DF if "lfp_RMS_filtered" in col]

In [48]:
filtered_RMS_column

['mPFC_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'vHPC_lfp_RMS_filtered']

In [49]:
for col in filtered_RMS_column:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS_interpolated".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: interpolate_signal(x))

mPFC_lfp_RMS_filtered
MD_lfp_RMS_filtered
LH_lfp_RMS_filtered
BLA_lfp_RMS_filtered
vHPC_lfp_RMS_filtered


In [50]:
# RMS_columns = [col for col in LFP_TRACES_DF if "RMS" in col and "filtered" not in col]
RMS_columns = [col for col in LFP_TRACES_DF if "interpolated" in col]

In [51]:
RMS_columns

['mPFC_lfp_RMS_interpolated',
 'MD_lfp_RMS_interpolated',
 'LH_lfp_RMS_interpolated',
 'BLA_lfp_RMS_interpolated',
 'vHPC_lfp_RMS_interpolated']

In [52]:
for col in RMS_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float32))

mPFC_lfp_RMS_interpolated
MD_lfp_RMS_interpolated
LH_lfp_RMS_interpolated
BLA_lfp_RMS_interpolated
vHPC_lfp_RMS_interpolated


In [53]:
LFP_TRACES_DF[col]

0     [0.24113564, 0.34435865, 0.55588114, 0.7394826...
1     [0.24113564, 0.34435865, 0.55588114, 0.7394826...
2     [0.90967715, 1.2532704, 1.5039245, 1.2560867, ...
3     [0.90967715, 1.2532704, 1.5039245, 1.2560867, ...
4     [0.2633122, 0.3874878, 0.33961287, -0.01047264...
                            ...                        
56    [0.2209081, 0.79025894, 1.1478113, 0.7128272, ...
57    [0.1772029, 0.0100303525, -0.15268426, -0.2028...
58    [0.1772029, 0.0100303525, -0.15268426, -0.2028...
59    [-1.0646816, -1.1657441, -1.1304898, -0.904861...
60    [-1.0646816, -1.1657441, -1.1304898, -0.904861...
Name: vHPC_lfp_RMS_interpolated, Length: 61, dtype: object

In [54]:
LFP_TRACES_DF[col]

0     [0.24113564, 0.34435865, 0.55588114, 0.7394826...
1     [0.24113564, 0.34435865, 0.55588114, 0.7394826...
2     [0.90967715, 1.2532704, 1.5039245, 1.2560867, ...
3     [0.90967715, 1.2532704, 1.5039245, 1.2560867, ...
4     [0.2633122, 0.3874878, 0.33961287, -0.01047264...
                            ...                        
56    [0.2209081, 0.79025894, 1.1478113, 0.7128272, ...
57    [0.1772029, 0.0100303525, -0.15268426, -0.2028...
58    [0.1772029, 0.0100303525, -0.15268426, -0.2028...
59    [-1.0646816, -1.1657441, -1.1304898, -0.904861...
60    [-1.0646816, -1.1657441, -1.1304898, -0.904861...
Name: vHPC_lfp_RMS_interpolated, Length: 61, dtype: object

In [55]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF if "lfp_mask" in col or "trace_filtered" in col], errors="ignore")

In [56]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_RMS_filtered', 'MD_lfp_RMS_filtered',
       'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered', 'vHPC_lfp_RMS_filtered',
       'mPFC_lfp_RMS_interpolated', 'MD_lfp_RMS_interpolated',
       'LH_lfp_RMS_interpolated', 'BLA_lfp_RMS_interpolated',
       'vHPC_lfp_RMS_interpolated'],
      dtype='object')

## Calculating phase of signals

In [57]:
from scipy.signal import butter, filtfilt, hilbert

- Filtering for theta and gamma

In [58]:
fs = 1000
order=4

In [59]:
freq_band = [4, 12]
b, a = butter(order, freq_band, fs=fs, btype='band')

In [60]:
for col in RMS_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_theta_band".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: filtfilt(b, a, x, padtype=None).astype(np.float32))

mPFC_lfp_RMS_interpolated
MD_lfp_RMS_interpolated
LH_lfp_RMS_interpolated
BLA_lfp_RMS_interpolated
vHPC_lfp_RMS_interpolated


In [61]:
LFP_TRACES_DF[updated_column]

0     [0.18621668, 0.19521728, 0.20376085, 0.2118085...
1     [0.18621668, 0.19521728, 0.20376085, 0.2118085...
2     [0.2616766, 0.2322801, 0.20183444, 0.17042315,...
3     [0.2616766, 0.2322801, 0.20183444, 0.17042315,...
4     [0.022297505, 0.029372353, 0.03709691, 0.04545...
                            ...                        
56    [-0.28977153, -0.28415498, -0.2780504, -0.2714...
57    [0.19069691, 0.17287906, 0.1543593, 0.13518454...
58    [0.19069691, 0.17287906, 0.1543593, 0.13518454...
59    [-0.46137807, -0.41177154, -0.36049426, -0.307...
60    [-0.46137807, -0.41177154, -0.36049426, -0.307...
Name: vHPC_theta_band, Length: 61, dtype: object

In [62]:
freq_band = [30, 50]
b, a = butter(order, freq_band, fs=fs, btype='band')

In [63]:
for col in RMS_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_gamma_band".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: filtfilt(b, a, x, padtype=None).astype(np.float32))

mPFC_lfp_RMS_interpolated
MD_lfp_RMS_interpolated
LH_lfp_RMS_interpolated
BLA_lfp_RMS_interpolated
vHPC_lfp_RMS_interpolated


- Calculating the phase

In [64]:
band_columns = [col for col in LFP_TRACES_DF if "band" in col]

In [65]:
band_columns

['mPFC_theta_band',
 'MD_theta_band',
 'LH_theta_band',
 'BLA_theta_band',
 'vHPC_theta_band',
 'mPFC_gamma_band',
 'MD_gamma_band',
 'LH_gamma_band',
 'BLA_gamma_band',
 'vHPC_gamma_band']

In [66]:
for col in band_columns:
    print(col)
    brain_region = col.replace("_band", "")
    updated_column = "{}_phase".format(brain_region)
    print(updated_column)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: np.angle(hilbert(x), deg=False).astype(np.float32))

mPFC_theta_band
mPFC_theta_phase
MD_theta_band
MD_theta_phase
LH_theta_band
LH_theta_phase
BLA_theta_band
BLA_theta_phase
vHPC_theta_band
vHPC_theta_phase
mPFC_gamma_band
mPFC_gamma_phase
MD_gamma_band
MD_gamma_phase
LH_gamma_band
LH_gamma_phase
BLA_gamma_band
BLA_gamma_phase
vHPC_gamma_band
vHPC_gamma_phase


In [67]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_RMS_filtered', 'MD_lfp_RMS_filtered',
       'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered', 'vHPC_lfp_RMS_filtered',
       'mPFC_lfp_RMS_interpolated', 'MD_lfp_RMS_interpolated',
       'LH_lfp_RMS_interpolated', 'BLA_lfp_RMS_interpolated',
       'vHPC_lfp_RMS_interpolated', 'mPFC_theta_band', 'MD_theta_band',
       'LH_theta_band', 'BLA_theta_band', 'vHPC_theta_band', 'mPFC_gamma_band',
       'MD_gamma_band', 'LH_gamma_band', 'BLA_gamma_band', 'vHPC_gamma_band',
       'mPFC_theta_phase', 'MD_theta_phase', 'LH_theta_phase',
       'BLA_theta_phase', 'vHPC_theta_phase', 'mPFC_gamma_phase',
       '

# Power Calcuation

- Getting the column name of all the traces

In [68]:
input_columns = [col for col in LFP_TRACES_DF.columns if "lfp_RMS_filtered" in col]

In [69]:
input_columns

['mPFC_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'vHPC_lfp_RMS_filtered']

- Calcuating the power at each frequency band

In [70]:
LFP_TRACES_DF[col].iloc[0]

array([-2.5841529e-02, -1.4202076e-02, -2.8395660e-03, ...,
        1.4404488e-05,  1.5425836e-06, -2.0803222e-17], dtype=float32)

In [71]:
for col in input_columns:
    # brain_region = col.split("_")[0]
    brain_region = col.replace("_lfp", "")
    print(brain_region)

    # Define column names
    multitaper_col = f"{brain_region}_power_multitaper"
    connectivity_col = f"{brain_region}_power_connectivity"
    frequencies_col = f"{brain_region}_power_frequencies"
    power_col = f"{brain_region}_power_all_frequencies_all_windows"
    
    try:
        # Apply Multitaper function to the lfp_trace column
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF[col].apply(
            lambda x: Multitaper(
                time_series=x, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT,
                time_window_duration=TIME_WINDOW_DURATION, 
                time_window_step=TIME_WINDOW_STEP
            )
        )

        # Apply Connectivity function to the multitaper column
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and power functions to the connectivity column
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.power().squeeze()
        )
        
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[power_col].apply(lambda x: x.astype(np.float32))
            
        # Removing unnecessary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")
    
    except Exception as e: 
        print(e)

mPFC_RMS_filtered
MD_RMS_filtered
LH_RMS_filtered
BLA_RMS_filtered
vHPC_RMS_filtered


- Getting the timestamps of the power

In [72]:
LFP_TRACES_DF["power_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])
# .iloc[0][500:-500:500].shape

- Making sure that the timestamps for power makes sense with shape and values

In [73]:
LFP_TRACES_DF["power_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6828,)
3    (6828,)
4    (6833,)
Name: power_timestamps, dtype: object

In [74]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [75]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340])

In [76]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000])

- Checking if the right frequencies are being used

In [77]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col]].head()

,mPFC_RMS_filtered_power_frequencies,MD_RMS_filtered_power_frequencies,LH_RMS_filtered_power_frequencies,BLA_RMS_filtered_power_frequencies,vHPC_RMS_filtered_power_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [78]:
LFP_TRACES_DF["power_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [79]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col], errors="ignore")

In [80]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,LH_gamma_phase,BLA_gamma_phase,vHPC_gamma_phase,mPFC_RMS_filtered_power_all_frequencies_all_windows,MD_RMS_filtered_power_all_frequencies_all_windows,LH_RMS_filtered_power_all_frequencies_all_windows,BLA_RMS_filtered_power_all_frequencies_all_windows,vHPC_RMS_filtered_power_all_frequencies_all_windows,power_timestamps,power_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-0.62610334, 0.039095968, 0.21467249, 0.60808...","[-0.8748, -0.4461616, -0.28834346, 0.007925385...","[-3.0228662, -2.0309567, -1.6685622, -1.346504...","[[0.041817404, 0.08001584, 0.089131445, 0.0703...","[[0.006807263, 0.018917628, 0.023094367, 0.026...","[[0.020440647, 0.051639214, 0.062006716, 0.059...","[[0.0428919, 0.02788801, 0.051582906, 0.053326...","[[0.008977723, 0.005360422, 0.015187492, 0.014...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-0.62610334, 0.039095968, 0.21467249, 0.60808...","[-0.8748, -0.4461616, -0.28834346, 0.007925385...","[-3.0228662, -2.0309567, -1.6685622, -1.346504...","[[0.041817404, 0.08001584, 0.089131445, 0.0703...","[[0.006807263, 0.018917628, 0.023094367, 0.026...","[[0.020440647, 0.051639214, 0.062006716, 0.059...","[[0.0428919, 0.02788801, 0.051582906, 0.053326...","[[0.008977723, 0.005360422, 0.015187492, 0.014...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-0.7881294, -0.28287637, -0.14779639, 0.17571...","[-2.1103802, -1.5603422, -1.2369094, -0.990382...","[-2.73681, -1.8883948, -1.543798, -1.2569945, ...","[[0.00027959025, 0.0005572859, 0.0021156531, 0...","[[0.010760473, 0.0112870205, 0.00968689, 0.003...","[[0.014777711, 0.015996255, 0.014629032, 0.005...","[[0.00058458163, 0.0018011965, 0.0063364967, 0...","[[0.0012987665, 0.0016842749, 0.005790464, 0.0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-0.7881294, -0.28287637, -0.14779639, 0.17571...","[-2.1103802, -1.5603422, -1.2369094, -0.990382...","[-2.73681, -1.8883948, -1.543798, -1.2569945, ...","[[0.00027959025, 0.0005572859, 0.0021156531, 0...","[[0.010760473, 0.0112870205, 0.00968689, 0.003...","[[0.014777711, 0.015996255, 0.014629032, 0.005...","[[0.00058458163, 0.0018011965, 0.0063364967, 0...","[[0.0012987665, 0.0016842749, 0.005790464, 0.0...","[10000, 20000, 30000, 40000, 50000, 60000, 

# Getting all the pairs

In [81]:
input_columns

['mPFC_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'vHPC_lfp_RMS_filtered']

In [82]:
all_suffixes = set(["_".join(col.split("_")[1:]) for col in input_columns])

In [83]:
all_suffixes

{'lfp_RMS_filtered'}

In [84]:
brain_region_pairs = generate_pairs(sorted(list(set([col.split("lfp")[0].strip("_") for col in sorted(input_columns)]))))

In [85]:
brain_region_pairs

[('BLA', 'LH'),
 ('BLA', 'MD'),
 ('BLA', 'mPFC'),
 ('BLA', 'vHPC'),
 ('LH', 'MD'),
 ('LH', 'mPFC'),
 ('LH', 'vHPC'),
 ('MD', 'mPFC'),
 ('MD', 'vHPC'),
 ('mPFC', 'vHPC')]

In [86]:
for first_region, second_region in brain_region_pairs:
    for suffix in all_suffixes:
        suffix_for_name = suffix.replace("lfp", "").strip("_")
        region_1 = "_".join([first_region, suffix])
        region_2 = "_".join([second_region, suffix])
         # Define base name for pair
        pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}_{suffix_for_name}"
        print(pair_base_name)

BLA_LH_RMS_filtered
BLA_MD_RMS_filtered
BLA_mPFC_RMS_filtered
BLA_vHPC_RMS_filtered
LH_MD_RMS_filtered
LH_mPFC_RMS_filtered
LH_vHPC_RMS_filtered
MD_mPFC_RMS_filtered
MD_vHPC_RMS_filtered
mPFC_vHPC_RMS_filtered


## Coherece Calculation

In [87]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF if "coherence" in col or "trace_filtered" in col], errors="ignore")

In [88]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_RMS_filtered', 'MD_lfp_RMS_filtered',
       'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered', 'vHPC_lfp_RMS_filtered',
       'mPFC_lfp_RMS_interpolated', 'MD_lfp_RMS_interpolated',
       'LH_lfp_RMS_interpolated', 'BLA_lfp_RMS_interpolated',
       'vHPC_lfp_RMS_interpolated', 'mPFC_theta_band', 'MD_theta_band',
       'LH_theta_band', 'BLA_theta_band', 'vHPC_theta_band', 'mPFC_gamma_band',
       'MD_gamma_band', 'LH_gamma_band', 'BLA_gamma_band', 'vHPC_gamma_band',
       'mPFC_theta_phase', 'MD_theta_phase', 'LH_theta_phase',
       'BLA_theta_phase', 'vHPC_theta_phase', 'mPFC_gamma_phase',
       '

- Calculating the coherence

In [89]:
column_to_recording_dict = recursive_defaultdict()

In [90]:
for first_region, second_region in brain_region_pairs:
    for suffix in all_suffixes:
        suffix_for_name = suffix.replace("lfp", "").strip("_")
        region_1 = "_".join([first_region, suffix])
        region_2 = "_".join([second_region, suffix])
         # Define base name for pair
        pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}_{suffix_for_name}"
        print(region_1)
        print(region_2)
        print(pair_base_name)

        try:
            # Define column names
            multitaper_col = f"{pair_base_name}_coherence_multitaper"
            connectivity_col = f"{pair_base_name}_coherence_connectivity"
            frequencies_col = f"{pair_base_name}_coherence_frequencies"
            coherence_col = f"{pair_base_name}_coherence_all_frequencies_all_windows"

            for index, row in LFP_TRACES_DF.drop_duplicates(subset=["recording"]).iterrows():
                recording = row["recording"]
                print("\t", recording)
                
                
                # Apply Multitaper function
                multitaper = Multitaper(
                        time_series=np.array([row[region_1], row[region_2]]).T, 
                        sampling_frequency=RESAMPLE_RATE, 
                        time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                        time_window_step=TIME_WINDOW_STEP, 
                        time_window_duration=TIME_WINDOW_DURATION)

                # Apply Connectivity function
                connectivity = Connectivity.from_multitaper(multitaper)

                column_to_recording_dict[coherence_col][recording] = connectivity.coherence_magnitude()[:,:,0,1]

                column_to_recording_dict[coherence_col][recording] =  column_to_recording_dict[coherence_col][recording][:,:62].astype(np.float32)
                # Apply frequencies and coherence functions
                column_to_recording_dict[frequencies_col][recording] = connectivity.frequencies[:62]

        except Exception as e: 
            print(e)


BLA_lfp_RMS_filtered
LH_lfp_RMS_filtered
BLA_LH_RMS_filtered
	 20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged
	 20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged
	 20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged
	 20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged
	 20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged
	 20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged
	 20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged
	 20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged
	 20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged
	 20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged
	 20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged
	 20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged
	 20230618_100636_standard_comp_to_omission_D2_subj

In [91]:
for key, value in column_to_recording_dict.items():
    LFP_TRACES_DF[key] = LFP_TRACES_DF["recording"].map(value)

- Getting the timestamps of the coherence

In [92]:
LFP_TRACES_DF["coherence_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for coherence makes sense with shape and values

In [93]:
LFP_TRACES_DF["coherence_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6828,)
3    (6828,)
4    (6833,)
Name: coherence_timestamps, dtype: object

In [94]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 62)

In [95]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340])

In [96]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000])

- Checking if the right frequencies are being used

In [97]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col]].head()

,BLA_LH_RMS_filtered_coherence_frequencies,BLA_MD_RMS_filtered_coherence_frequencies,BLA_mPFC_RMS_filtered_coherence_frequencies,BLA_vHPC_RMS_filtered_coherence_frequencies,LH_MD_RMS_filtered_coherence_frequencies,LH_mPFC_RMS_filtered_coherence_frequencies,LH_vHPC_RMS_filtered_coherence_frequencies,MD_mPFC_RMS_filtered_coherence_frequencies,MD_vHPC_RMS_filtered_coherence_frequencies,mPFC_vHPC_RMS_filtered_coherence_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [98]:
LFP_TRACES_DF["coherence_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [99]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_RMS_filtered', 'MD_lfp_RMS_filtered',
       'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered', 'vHPC_lfp_RMS_filtered',
       'mPFC_lfp_RMS_interpolated', 'MD_lfp_RMS_interpolated',
       'LH_lfp_RMS_interpolated', 'BLA_lfp_RMS_interpolated',
       'vHPC_lfp_RMS_interpolated', 'mPFC_theta_band', 'MD_theta_band',
       'LH_theta_band', 'BLA_theta_band', 'vHPC_theta_band', 'mPFC_gamma_band',
       'MD_gamma_band', 'LH_gamma_band', 'BLA_gamma_band', 'vHPC_gamma_band',
       'mPFC_theta_phase', 'MD_theta_phase', 'LH_theta_phase',
       'BLA_theta_phase', 'vHPC_theta_phase', 'mPFC_gamma_phase',
       '

In [100]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col], errors="ignore")

In [101]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_RMS_filtered', 'MD_lfp_RMS_filtered',
       'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered', 'vHPC_lfp_RMS_filtered',
       'mPFC_lfp_RMS_interpolated', 'MD_lfp_RMS_interpolated',
       'LH_lfp_RMS_interpolated', 'BLA_lfp_RMS_interpolated',
       'vHPC_lfp_RMS_interpolated', 'mPFC_theta_band', 'MD_theta_band',
       'LH_theta_band', 'BLA_theta_band', 'vHPC_theta_band', 'mPFC_gamma_band',
       'MD_gamma_band', 'LH_gamma_band', 'BLA_gamma_band', 'vHPC_gamma_band',
       'mPFC_theta_phase', 'MD_theta_phase', 'LH_theta_phase',
       'BLA_theta_phase', 'vHPC_theta_phase', 'mPFC_gamma_phase',
       '

In [102]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,BLA_mPFC_RMS_filtered_coherence_all_frequencies_all_windows,BLA_vHPC_RMS_filtered_coherence_all_frequencies_all_windows,LH_MD_RMS_filtered_coherence_all_frequencies_all_windows,LH_mPFC_RMS_filtered_coherence_all_frequencies_all_windows,LH_vHPC_RMS_filtered_coherence_all_frequencies_all_windows,MD_mPFC_RMS_filtered_coherence_all_frequencies_all_windows,MD_vHPC_RMS_filtered_coherence_all_frequencies_all_windows,mPFC_vHPC_RMS_filtered_coherence_all_frequencies_all_windows,coherence_timestamps,coherence_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[0.895295, 0.75152117, 0.7654758, 0.5750287, ...","[[0.85538274, 0.9473491, 0.85133094, 0.8886854...","[[0.8989649, 0.9760324, 0.97991437, 0.9659002,...","[[0.98758525, 0.9969371, 0.99350476, 0.9860859...","[[0.5242202, 0.6692347, 0.6810993, 0.7297231, ...","[[0.9551989, 0.96587896, 0.9731124, 0.94918567...","[[0.8203178, 0.6313364, 0.6094348, 0.6193265, ...","[[0.63334197, 0.71688265, 0.6821726, 0.7018544...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[0.895295, 0.75152117, 0.7654758, 0.5750287, ...","[[0.85538274, 0.9473491, 0.85133094, 0.8886854...","[[0.8989649, 0.9760324, 0.97991437, 0.9659002,...","[[0.98758525, 0.9969371, 0.99350476, 0.9860859...","[[0.5242202, 0.6692347, 0.6810993, 0.7297231, ...","[[0.9551989, 0.96587896, 0.9731124, 0.94918567...","[[0.8203178, 0.6313364, 0.6094348, 0.6193265, ...","[[0.63334197, 0.71688265, 0.6821726, 0.7018544...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[0.49185678, 0.48341042, 0.83400226, 0.549778...","[[0.94447166, 0.9370585, 0.97242934, 0.8198627...","[[0.99695826, 0.9975801, 0.9912695, 0.9616406,...","[[0.05826523, 0.2382561, 0.29821226, 0.2188783...","[[0.018408593, 0.818058, 0.11358966, 0.4236967...","[[0.07472135, 0.21970199, 0.29790184, 0.337977...","[[0.0065454636, 0.7808023, 0.14767082, 0.59176...","[[0.34496546, 0.3153885, 0.76019776, 0.8979203...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[0.49185678, 0.48341042, 0.83400226, 0.549778...","[[0.94447166, 0.9370585, 0.97242934, 0.8198627...","[[0.99695826, 0.9975801, 0.9912695, 0.9616406,...","[[0.05826523, 0.2382561, 0.29821226, 0.2188783...","[[0.018408593, 0.818058, 0.11358966, 0.4236967...","[[0.07472135, 0.2197

# Calculate Granger's

In [103]:
column_to_recording_dict = recursive_defaultdict()

In [104]:
for first_region, second_region in brain_region_pairs:
    for suffix in all_suffixes:
        region_1 = "_".join([first_region, suffix])
        region_2 = "_".join([second_region, suffix])
        region_1_base_name = region_1.split('_')[0]
        region_2_base_name = region_2.split('_')[0]
        pair_base_name = f"{region_1_base_name}_{region_2_base_name}"
        print(pair_base_name)

        try:
            # Define column names
            frequencies_col = f"{pair_base_name}_granger_frequencies"
            granger_1_2_col = f"{region_1_base_name}_{region_2_base_name}_granger_all_frequencies_all_windows"
            granger_2_1_col = f"{region_2_base_name}_{region_1_base_name}_granger_all_frequencies_all_windows"
            
            for index, row in LFP_TRACES_DF.drop_duplicates(subset=["recording"]).iterrows():
                recording = row["recording"]
                print("\t", recording)
                # Apply Multitaper function
                multitaper = Multitaper(
                        time_series=np.array([row[region_1], row[region_2]]).T, 
                        sampling_frequency=RESAMPLE_RATE, 
                        time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                        time_window_step=TIME_WINDOW_STEP, 
                        time_window_duration=TIME_WINDOW_DURATION)

                # Apply Connectivity function
                connectivity = Connectivity.from_multitaper(multitaper)

                # Apply frequencies and coherence functions
                column_to_recording_dict[frequencies_col][recording] = connectivity.frequencies[:62]
                
                column_to_recording_dict[granger_1_2_col][recording] = connectivity.pairwise_spectral_granger_prediction()[:,:,0,1]
                
                column_to_recording_dict[granger_2_1_col][recording] = connectivity.pairwise_spectral_granger_prediction()[:,:,1,0]

                column_to_recording_dict[granger_1_2_col][recording] = column_to_recording_dict[granger_1_2_col][recording].astype(np.float32)[:, :62]
                column_to_recording_dict[granger_2_1_col][recording] = column_to_recording_dict[granger_2_1_col][recording].astype(np.float32)[:, :62]
        
        except Exception as e: 
            print(e)

BLA_LH
	 20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged


Maximum iterations reached. 6012 of 6828 converged
Maximum iterations reached. 6012 of 6828 converged


	 20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged


Maximum iterations reached. 5138 of 6828 converged
Maximum iterations reached. 5138 of 6828 converged


	 20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged


...

Maximum iterations reached. 6002 of 6842 converged
Maximum iterations reached. 6002 of 6842 converged


	 20230630_115506_standard_comp_to_novel_agent_D3_subj_1-2vs1-4and2-2_merged


Maximum iterations reached. 5768 of 6855 converged
Maximum iterations reached. 5768 of 6855 converged


	 20230630_115506_standard_comp_to_novel_agent_D3_subj_1-4vs1-2and2-1_merged


Maximum iterations reached. 6124 of 6855 converged
Maximum iterations reached. 6124 of 6855 converged


In [105]:
for key, value in column_to_recording_dict.items():
    LFP_TRACES_DF[key] = LFP_TRACES_DF["recording"].map(value)

- Getting the timestamps of the granger

In [106]:
LFP_TRACES_DF["granger_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for granger makes sense with shape and values

In [107]:
LFP_TRACES_DF["granger_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6828,)
3    (6828,)
4    (6833,)
Name: granger_timestamps, dtype: object

In [108]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 62)

In [109]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340])

In [110]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000])

- Checking if the right frequencies are being used

In [111]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col]].head()

,BLA_LH_granger_frequencies,BLA_MD_granger_frequencies,BLA_mPFC_granger_frequencies,BLA_vHPC_granger_frequencies,LH_MD_granger_frequencies,LH_mPFC_granger_frequencies,LH_vHPC_granger_frequencies,MD_mPFC_granger_frequencies,MD_vHPC_granger_frequencies,mPFC_vHPC_granger_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [112]:
LFP_TRACES_DF["granger_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [113]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col], errors="ignore")

In [114]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_RMS_filtered', 'MD_lfp_RMS_filtered',
       'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered', 'vHPC_lfp_RMS_filtered',
       'mPFC_lfp_RMS_interpolated', 'MD_lfp_RMS_interpolated',
       'LH_lfp_RMS_interpolated', 'BLA_lfp_RMS_interpolated',
       'vHPC_lfp_RMS_interpolated', 'mPFC_theta_band', 'MD_theta_band',
       'LH_theta_band', 'BLA_theta_band', 'vHPC_theta_band', 'mPFC_gamma_band',
       'MD_gamma_band', 'LH_gamma_band', 'BLA_gamma_band', 'vHPC_gamma_band',
       'mPFC_theta_phase', 'MD_theta_phase', 'LH_theta_phase',
       'BLA_theta_phase', 'vHPC_theta_phase', 'mPFC_gamma_phase',
       '

In [115]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,LH_vHPC_granger_all_frequencies_all_windows,vHPC_LH_granger_all_frequencies_all_windows,MD_mPFC_granger_all_frequencies_all_windows,mPFC_MD_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[1.2923173, 0.62806845, 0.28610364, 0.6162430...","[[0.4101196, 0.23832376, 0.023698151, 0.147102...","[[1.7712247, 1.6961608, 1.5926156, 0.9402654, ...","[[1.632409, 0.92598146, 1.146866, 0.7823483, 0...","[[2.1441872, 0.4484717, 0.31936106, 0.59183514...","[[0.18993315, 0.17902073, 0.019536048, 0.09441...","[[0.38200194, 0.26012245, 0.1328726, 0.1788294...","[[0.00093113154, 0.21856883, 0.21809189, 0.603...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[1.2923173, 0.62806845, 0.28610364, 0.6162430...","[[0.4101196, 0.23832376, 0.023698151, 0.147102...","[[1.7712247, 1.6961608, 1.5926156, 0.9402654, ...","[[1.632409, 0.92598146, 1.146866, 0.7823483, 0...","[[2.1441872, 0.4484717, 0.31936106, 0.59183514...","[[0.18993315, 0.17902073, 0.019536048, 0.09441...","[[0.38200194, 0.26012245, 0.1328726, 0.1788294...","[[0.00093113154, 0.21856883, 0.21809189, 0.603...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[2.609014, 1.3195373, 0.6954608, 0.19390115, ...","[[1.4132965, 0.8919425, 0.16531233, 0.28616, 0...","[[0.07916975, 0.072599195, 0.121737845, 0.0310...","[[1.4273785, 0.29512075, 0.37303713, 0.1411382...","[[1.6550303, 1.5495375, 0.8795545, 0.21966118,...","[[0.5943993, 0.3949601, 0.17542279, 0.27749178...","[[1.060304, 2.445954, 0.57600904, 0.31814077, ...","[[nan, 0.77303505, 1.7006428, 0.66405064, 0.34...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[2.609014, 1.3195373, 0.6954608, 0.19390115, ...","[[1.4132965, 0.8919425, 0.16531233, 0.28616, 0...","[[0.07916975, 0.072599195, 0.121737845, 0.0310...","[[1.4273785, 0.29512075, 0.37303713, 0.1411382...","[[1.6550303, 1.5495375, 0.8795545, 0.21966118,...","[[0.5943993, 0.3949601, 0.17542279, 0.27749178...","[[1.060304, 2.445954, 0.57600904, 0.31814077, ...","[[nan, 0.77303505, 1.7006428, 0.664050

# Saving the spectral analysis

In [116]:
# LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")
LFP_TRACES_DF.to_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")
# LFP_TRACES_DF.to_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/rce_pilot_2_02_spectral_granger.pkl")

In [117]:
print("all done!")

all done!
